In [261]:
import os
import sys
import warnings
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.display.max_rows = 50
pd.options.display.max_columns = None

# Load preprocessed data

In [262]:
df_sales = pd.read_csv('sales_data_preprocessed.csv', parse_dates=['invoice_date'])
df_weather = pd.read_csv('weather_data.csv', parse_dates=['date'])
df_weather['date'] = df_weather['date'].dt.tz_convert(None)


# Feature Engineering

In [263]:
# Start by creating a copy of the sales dataframe
df_features = df_sales.copy()
df_features['sales'] = df_features['price'] * df_features['quantity']

## Separating train and test

In [264]:
# Test set will be the last 3 month 2023 and first month 2024
cutoff_date = pd.Timestamp('2023-10-01')
print(f"Train-test split date: {cutoff_date}")

# Create a flag for train/test split
df_features['is_test'] = df_features['invoice_date'] >= cutoff_date
print(f"Training df_features: {len(df_features[~df_features['is_test']])} rows")
print(f"Testing df_features: {len(df_features[df_features['is_test']])} rows")

# Merge weather df_features
df_features = pd.merge(
    left=df_features,
    right=df_weather,
    left_on=['invoice_date'],
    right_on=['date'],
    how='left'
)

df_features['temperature'] = (df_features['temperature_2m_max'] + df_features['temperature_2m_min']) / 2

df_features.head()

Train-test split date: 2023-10-01 00:00:00
Training df_features: 130565 rows
Testing df_features: 16106 rows


,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,sales,is_test,date,temperature_2m_max,temperature_2m_min,rain_sum,temperature
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity,130.75,False,2021-01-01,13.806,7.756,4.2,10.781
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul,4801.28,False,2021-01-01,13.806,7.756,4.2,10.781
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity,2700.72,False,2021-01-01,13.806,7.756,4.2,10.781
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul,83.68,False,2021-01-01,13.806,7.756,4.2,10.781
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park,300.08,False,2021-01-01,13.806,7.756,4.2,10.781


In [265]:
df_weather['mean temperature'] = (df_weather['temperature_2m_max'] + df_weather['temperature_2m_min']) / 2
df_weather.describe()

,date,temperature_2m_max,temperature_2m_min,rain_sum,mean temperature
count,1111,1111.000000,1111.000000,1111.000000,1111.000000
mean,2022-07-10 00:00:00,18.993759,11.731968,2.112961,15.362863
min,2021-01-01 00:00:00,0.906000,-6.444000,0.000000,-1.894000
25%,2021-10-05 12:00:00,13.006001,6.556000,0.000000,9.931000
50%,2022-07-10 00:00:00,18.806000,11.506001,0.000000,15.031000
75%,2023-04-13 12:00:00,25.556000,17.606000,1.500000,21.731000
max,2024-01-16 00:00:00,38.806000,24.556000,57.700000,30.106000
std,NaN,7.890963,6.806037,5.201321,7.236332


## Add date and holidays related features

In [266]:
# Date features
df_features['year'] = df_features['invoice_date'].dt.year
df_features['month'] = df_features['invoice_date'].dt.month
df_features['day'] = df_features['invoice_date'].dt.day
df_features['day_of_week'] = df_features['invoice_date'].dt.dayofweek
df_features['is_weekend'] = df_features['day_of_week'].apply(
    lambda x: 1 if x > 5 else 0
)
df_features['quarter'] = df_features['invoice_date'].dt.quarter

# For simplicity, considering some majot holidays in turkey
holidays = [
    "2021-03-29",  # Eid al-Fitr (Ramazan Bayramı)
    "2021-04-23",  # National Sovereignty & Children’s Day
    "2021-05-19",  # Atatürk, Youth & Sports Day
    "2021-06-06",  # Eid al-Adha (Kurban Bayramı)
    "2021-08-30",  # Victory Day
    "2021-10-29",  # Republic Day

    "2022-03-29",
    "2022-04-24",
    "2021-05-19",
    "2022-06-06",
    "2022-08-30",
    "2022-10-29",

    "2023-03-29",
    "2023-04-24",
    "2021-05-19",
    "2023-06-06",
    "2023-08-30",
    "2023-10-29",
    ""
]
holidays = pd.to_datetime(holidays)
df_features["is_holiday"] = df_features['invoice_date'].isin(holidays).astype(int)
df_features.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,sales,is_test,date,temperature_2m_max,temperature_2m_min,rain_sum,temperature,year,month,day,day_of_week,is_weekend,quarter,is_holiday
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity,130.75,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul,4801.28,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity,2700.72,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul,83.68,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park,300.08,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0


## Weather-based features

In [267]:
# Temperature categories (using quantiles for evven distribution)
tem_pins = [-np.inf, 5, 15, 25, np.inf]
temp_labels = ["Cold", "Cool", 'Warm', 'Hot']
df_features['temp_category'] = pd.cut(
    df_features['temperature'], 
    bins=tem_pins,
    labels=temp_labels
)

# Humidity levels
rain_bins = [-np.inf, 1.4, 5, np.inf]
rain_labels = ["Low", "Medium", "High"]

df_features["rain_level"] = pd.cut(
    df_features["rain_sum"], bins=rain_bins, labels=rain_labels
)

df_features.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,sales,is_test,date,temperature_2m_max,temperature_2m_min,rain_sum,temperature,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category,rain_level
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity,130.75,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,Cool,Medium
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul,4801.28,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,Cool,Medium
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity,2700.72,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,Cool,Medium
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul,83.68,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,Cool,Medium
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park,300.08,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,Cool,Medium


In [268]:
df_features = pd.get_dummies(
    df_features, columns=["temp_category", "rain_level"], drop_first=True
)

df_features.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,sales,is_test,date,temperature_2m_max,temperature_2m_min,rain_sum,temperature,year,month,day,day_of_week,is_weekend,quarter,is_holiday,temp_category_Cool,temp_category_Warm,temp_category_Hot,rain_level_Medium,rain_level_High
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity,130.75,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,True,False,False,True,False
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul,4801.28,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,True,False,False,True,False
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity,2700.72,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,True,False,False,True,False
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul,83.68,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,True,False,False,True,False
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park,300.08,False,2021-01-01,13.806,7.756,4.2,10.781,2021,1,1,4,0,1,0,True,False,False,True,False


### Last n day sales

In [269]:
# Sort df_features by date, and category for proper lag feature creation
df_features = df_features.sort_values(['invoice_date', 'shopping_mall', 'category'])

# Create an identifier for store-item combinations for grouping
df_features['shopping_mall_category'] = df_features['shopping_mall'] + " " + df_features['category']

In [270]:
df_features_shopping_mall_category = df_features.groupby(['invoice_date', 'shopping_mall', 'category']).agg(
    Total_Sales=('quantity', 'sum'),
    Revenue=('sales', 'sum')
).reset_index(['invoice_date','shopping_mall', 'category'])
df_features_shopping_mall_category["shopping_mall_category"] = df_features_shopping_mall_category['shopping_mall'] + " " + df_features_shopping_mall_category['category']
df_features_shopping_mall_category

,invoice_date,shopping_mall,category,Total_Sales,Revenue,shopping_mall_category
0,2021-01-01,Cevahir AVM,Clothing,3,2700.720000,Cevahir AVM Clothing
1,2021-01-01,Cevahir AVM,Food & Beverage,3,47.070000,Cevahir AVM Food & Beverage
2,2021-01-01,Cevahir AVM,Souvenir,1,11.730000,Cevahir AVM Souvenir
3,2021-01-01,Cevahir AVM,Toys,4,573.440000,Cevahir AVM Toys
4,2021-01-01,Emaar Square Mall,Books,2,60.600000,Emaar Square Mall Books
...,...,...,...,...,...,...
61486,2024-01-15,Zorlu Center,Cosmetics,5,203.300000,Zorlu Center Cosmetics
61487,2024-01-15,Zorlu Center,Food & Beverage,9,214.430000,Zorlu Center Food & Beverage
61488,2024-01-15,Zorlu Center,Shoes,10,8269.777495,Zorlu Center Shoes
61489,2024-01-15,Zorlu Center,Souvenir,8,281.520000,Zorlu Center Souvenir


In [271]:
# Create lag features for df_features_shopping_mall_category
for n in [1, 7, 14, 28]:
    df_features_shopping_mall_category[f"sales_lag{n}"] = df_features_shopping_mall_category.groupby('shopping_mall_category')[
        "Revenue"
    ].transform(lambda x: x.shift(n))

## Rolling average features

In [272]:
# Create rolling average feeatures
for window in [7, 14, 28]:
    # Mean of last n days
    df_features_shopping_mall_category[f"sales_mean_{window}d"] = df_features_shopping_mall_category.groupby("shopping_mall_category")[
        "Revenue"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())

    # Min of last n days
    df_features_shopping_mall_category[f"sales_min_{window}d"] = df_features_shopping_mall_category.groupby("shopping_mall_category")[
        "Revenue"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).min())

    # Max of last n days
    df_features_shopping_mall_category[f"sales_max_{window}d"] = df_features_shopping_mall_category.groupby("shopping_mall_category")[
        "Revenue"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).max())

    # Standard deviation of last n days
    df_features_shopping_mall_category[f"sales_std_{window}d"] = df_features_shopping_mall_category.groupby("shopping_mall_category")[
        "Revenue"
    ].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).std())

In [273]:
df_features_shopping_mall_category.query("shopping_mall_category == 'Cevahir AVM Clothing'")

,invoice_date,shopping_mall,category,Total_Sales,Revenue,shopping_mall_category,sales_lag1,sales_lag7,sales_lag14,sales_lag28,sales_mean_7d,sales_min_7d,sales_max_7d,sales_std_7d,sales_mean_14d,sales_min_14d,sales_max_14d,sales_std_14d,sales_mean_28d,sales_min_28d,sales_max_28d,sales_std_28d
0,2021-01-01,Cevahir AVM,Clothing,3,2700.72,Cevahir AVM Clothing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2021-01-02,Cevahir AVM,Clothing,6,6001.60,Cevahir AVM Clothing,2700.72,NaN,NaN,NaN,2700.720000,2700.72,2700.72,NaN,2700.720000,2700.72,2700.72,NaN,2700.720000,2700.72,2700.72,NaN
104,2021-01-03,Cevahir AVM,Clothing,13,16504.40,Cevahir AVM Clothing,6001.60,NaN,NaN,NaN,4351.160000,2700.72,6001.60,2334.074632,4351.160000,2700.72,6001.60,2334.074632,4351.160000,2700.72,6001.60,2334.074632
155,2021-01-04,Cevahir AVM,Clothing,12,15004.00,Cevahir AVM Clothing,16504.40,NaN,NaN,NaN,8402.240000,2700.72,16504.40,7208.168956,8402.240000,2700.72,16504.40,7208.168956,8402.240000,2700.72,16504.40,7208.168956
203,2021-01-05,Cevahir AVM,Clothing,4,2400.64,Cevahir AVM Clothing,15004.00,NaN,NaN,NaN,10052.680000,2700.72,16504.40,6747.908953,10052.680000,2700.72,16504.40,6747.908953,10052.680000,2700.72,16504.40,6747.908953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61092,2024-01-10,Cevahir AVM,Clothing,1,300.08,Cevahir AVM Clothing,7502.00,11102.96,2100.56,8702.32,4201.120000,600.16,11102.96,3763.981005,5572.914286,600.16,14403.84,4780.625684,6612.477143,600.16,25206.72,5741.729405
61226,2024-01-12,Cevahir AVM,Clothing,3,2700.72,Cevahir AVM Clothing,300.08,1500.40,6001.60,10802.88,2657.851429,300.08,7502.00,2446.640232,5444.308571,300.08,14403.84,4903.852098,6312.397143,300.08,25206.72,5847.059716
61294,2024-01-13,Cevahir AVM,Clothing,4,3600.96,Cevahir AVM Clothing,2700.72,3600.96,900.24,5701.52,2829.325714,300.08,7502.00,2393.484574,5208.531429,300.08,14403.84,4954.092191,6023.034286,300.08,25206.72,5817.006386
61364,2024-01-14,Cevahir AVM,Clothing,3,1800.48,Cevahir AVM Clothing,3600.96,600.16,14403.84,6301.68,2829.325714,300.08,7502.00,2393.484574,5401.440000,300.08,14403.84,4824.307854,5948.014286,300.08,25206.72,5834.824245


## Store-level features

In [274]:
df_features_shopping_mall_category["store_date_mean"] = df_features_shopping_mall_category.groupby(["shopping_mall", "invoice_date"])[
    "Revenue"
].transform("mean")
df_features_shopping_mall_category["store_date_sum"] = df_features_shopping_mall_category.groupby(["shopping_mall", "invoice_date"])[
    "Revenue"
].transform("sum")

In [275]:
df_features_shopping_mall_category

,invoice_date,shopping_mall,category,Total_Sales,Revenue,shopping_mall_category,sales_lag1,sales_lag7,sales_lag14,sales_lag28,sales_mean_7d,sales_min_7d,sales_max_7d,sales_std_7d,sales_mean_14d,sales_min_14d,sales_max_14d,sales_std_14d,sales_mean_28d,sales_min_28d,sales_max_28d,sales_std_28d,store_date_mean,store_date_sum
0,2021-01-01,Cevahir AVM,Clothing,3,2700.720000,Cevahir AVM Clothing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,833.240000,3332.960000
1,2021-01-01,Cevahir AVM,Food & Beverage,3,47.070000,Cevahir AVM Food & Beverage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,833.240000,3332.960000
2,2021-01-01,Cevahir AVM,Souvenir,1,11.730000,Cevahir AVM Souvenir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,833.240000,3332.960000
3,2021-01-01,Cevahir AVM,Toys,4,573.440000,Cevahir AVM Toys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,833.240000,3332.960000
4,2021-01-01,Emaar Square Mall,Books,2,60.600000,Emaar Square Mall Books,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4239.805000,25438.830000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61486,2024-01-15,Zorlu Center,Cosmetics,5,203.300000,Zorlu Center Cosmetics,1016.50,1463.760000,1437.614999,1219.80,1028.117143,325.28,1504.42,413.130602,1127.068928,162.640000,3281.829998,785.176656,1095.018750,162.640000,3281.829998,665.029939,4003.589642,28025.127495
61487,2024-01-15,Zorlu Center,Food & Beverage,9,214.430000,Zorlu Center Food & Beverage,41.84,41.840000,83.680000,193.51,40.345714,10.46,94.14,28.118127,70.978571,5.230000,235.350000,76.555435,84.240357,5.230000,235.350000,71.417671,4003.589642,28025.127495
61488,2024-01-15,Zorlu Center,Shoes,10,8269.777495,Zorlu Center Shoes,42612.07,2400.680000,3601.020000,6001.70,17147.714286,2400.68,42612.07,13107.256357,12131.509107,2400.680000,42612.070000,10956.385920,11900.214464,718.807499,42612.070000,10117.374509,4003.589642,28025.127495
61489,2024-01-15,Zorlu Center,Souvenir,8,281.520000,Zorlu Center Souvenir,586.50,222.870000,328.440000,46.92,221.194286,46.92,586.50,173.512642,247.167857,46.920000,621.690000,194.136845,251.357143,35.190000,738.990000,190.039102,4003.589642,28025.127495


## Item-level features

In [276]:
# Mean and sum of item sales in the last 7 days
df_features_shopping_mall_category["category_date_mean"] = df_features_shopping_mall_category.groupby(["category", "invoice_date"])[
    "Revenue"
].transform("mean")
df_features_shopping_mall_category["category_date_sum"] = df_features_shopping_mall_category.groupby(["category", "invoice_date"])[
    "Revenue"
].transform("sum")

In [283]:
# Sample data: Daily sales
data = {'Sales': [200, 220, 210, 250, 240, 260, 270]}
df = pd.DataFrame(data)

# Applying Exponentially Weighted Moving Average (EWMA) with span=3
df['EWMA_Span_3'] = df['Sales'].ewm(span=3, adjust=False).mean()

# Display the DataFrame
print(df)

   Sales  EWMA_Span_3
0    200       200.00
1    220       210.00
2    210       210.00
3    250       230.00
4    240       235.00
5    260       247.50
6    270       258.75
